<a href="https://colab.research.google.com/github/eanasir/SSN-MEiL/blob/main/ssn_meil_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np


class Activation_fcn:

    def __init__(self):
        self.functions = {
            "linear":  self.linear,
            "sigmoid": self.sigmoid,
            "logistic": self.logistic,
            "tanh":    self.tanh,
            "relu":    self.relu,
        }

    def output(self, z, name, derivative=False):
        key = name.lower()

        if key not in self.functions:
            raise ValueError(f"Activation function '{name}' not found.")

        return self.functions[key](z, derivative)

    def linear(self, z, derivative=False):
        if derivative is True:
            out = np.ones_like(z)
            return out

        return z

    def sigmoid(self, z, derivative=False):
        s = 1.0 / (1.0 + np.exp(-z))

        if derivative is True:
            ds = s * (1.0 - s)
            return ds

        return s

    def tanh(self, z, derivative=False):
        t = np.tanh(z)

        if derivative is True:
            dt = 1.0 - (t * t)
            return dt

        return t

    def relu(self, z, derivative=False):
        if derivative is True:

            return np.where(z > 0, 1, 0).astype(z.dtype)

        return np.maximum(z, 0)

    def logistic(self, z, derivative=False):
        l = 1.0 / (1.0 + np.exp(-(z + 0.5)))

        if derivative is True:
            dl = l * (1.0 - l)
            return dl

        return l


In [ ]:
import sys
import numpy as np


class Loss_fcn:
  def __init__(self):
    self.functions = {
      'mse': self.mse,
      'bc_entropy': self.bc_entropy
      }
  def mse(self, expected, outputs, derivative=False):

    expected = np.asarray(expected, dtype=float)
    outputs = np.asarray(outputs, dtype=float)

    if not derivative:
      return np.power(expected - outputs, 2)
    else:
      return -2*(expected - outputs)

  def bc_entropy(self,expected,outputs,derivative = False):

    expected = np.asarray(expected, dtype=float)
    outputs = np.asarray(outputs, dtype=float)

    if not derivative:
      return expected * np.log(outputs) + (1- expected) * np.log(1 - outputs)
    else:
      return (1 - expected) / (1- outputs) - (expected / outputs)

In [ ]:
class Neural_network:

    def __init__(self, structure=None, init_weight='rand'):
        self.af = Activation_fcn()
        if structure:
            self.network = self.create_network(structure, init_weight=init_weight)

    def create_network(self, structure, init_weight='rand'):
        self.nnetwork = [structure[0]]
        for i in range(1, len(structure)):
            new_layer = {
                'weights': None,
                'activation_function': structure[i]['activation_function'],
                'activation_potential': None,
                'delta': None,
                'output': None
            }
            out_u = structure[i]['units']
            in_u  = structure[i-1]['units']
            match init_weight:
                case "rand":
                    new_layer["weights"] = np.random.randn(out_u, in_u)
                case "zero":
                    new_layer["weights"] = np.zeros((out_u, in_u))
                case "ones":
                    new_layer["weights"] = np.ones((out_u, in_u))
                case _:
                    print("init weight not recognized")
            self.nnetwork.append(new_layer)
        return self.nnetwork

    def forward_propagate(self, nnetwork, inputs):
        A = np.asarray(inputs).reshape(-1, 1)
        for i in range(1, len(nnetwork)):
            W = nnetwork[i]["weights"]
            Z = W @ A
            A = self.af.output(Z, nnetwork[i]['activation_function'])
            nnetwork[i]['activation_potential'] = Z
            nnetwork[i]['output'] = A
        return A

    def predict(self, nnetwork, inputs):

        X = np.asarray(inputs)
        if X.ndim == 1:
            return self.forward_propagate(nnetwork, X).ravel()
        elif X.ndim == 2:

            outs = []
            for row in X:
                y = self.forward_propagate(nnetwork, row).ravel()
                outs.append(y)
            return np.vstack(outs)
        else:
            raise ValueError("inputs must be 1D (in_units,) or 2D (n_samples, in_units)")


In [ ]:
structure = [{'type': 'input'
    ,
    'units': 1},
    {'type': 'dense'
    ,
    'units': 8,'activation_function': 'tanh'},
    {'type': 'dense'
    ,
    'units': 8,'activation_function': 'tanh'},
    {'type': 'dense'
    ,
    'units': 1,'activation_function': 'linear'}]
model = Neural_network(structure,'rand')
#print(model.network)

In [ ]:
structure = [
    {'units': 1},
    {'units': 3, 'activation_function': 'relu'},
    {'units': 1, 'activation_function': 'linear'}
]

model = Neural_network(structure=structure, init_weight='rand')
n = 1
X = np.linspace(-5, 5, n).reshape(-1, 1)
pred = model.predict(model.network, X)
print(pred)

[[10.09113234]]


In [ ]:
structure = [{'type': 'input','units': 1},
{'type': 'dense', 'units': 2,'activation_function': 'tanh'},
{'type': 'dense', 'units': 2,'activation_function': 'tanh'},
{'type': 'dense', 'units': 1,'activation_function': 'tanh'}]




network = model.create_network(structure, "ones")
n = 1
X = np.linspace(-5, 5, n).reshape(-1, 1)
predicted = model.predict(network, X)
print(predicted)

[[-0.95857383]]


In [ ]:
structure = [{'type': 'input','units': 1},
{'type': 'dense','units': 4,'activation_function': 'relu'},
{'type': 'dense', 'units': 4,'activation_function': 'relu'},
{'type': 'dense', 'units': 1,'activation_function': 'relu'}]





network = model.create_network(structure, "rand")
n = 1
X = np.linspace(1, 55, n).reshape(-1, 1)
predicted = model.predict(network, X)
print(predicted)

[[0.]]


In [ ]:
structure = [{'type': 'input','units': 1},
{'type': 'dense','units': 4,'activation_function': 'linear'},
{'type': 'dense', 'units': 4,'activation_function': 'linear'},
{'type': 'dense', 'units': 1,'activation_function': 'linear'}]





network = model.create_network(structure, "ones")
n = 5
X = np.linspace(5, 15, n).reshape(-1, 1)
predicted = model.predict(network, X)
print(predicted)

[[ 80.]
 [120.]
 [160.]
 [200.]
 [240.]]
